In [1]:
import os
from pandas import read_csv
import pandas as pd
from collections import Counter
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack
from matplotlib import pyplot as plt
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import (RandomForestClassifier,
                              GradientBoostingClassifier)
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import ADASYN, SMOTE, SMOTENC
from imblearn.pipeline import Pipeline
from scipy.sparse import hstack
import warnings
from sklearn.metrics import roc_curve, auc, f1_score, roc_auc_score, balanced_accuracy_score
warnings.filterwarnings("ignore")

In [2]:
data_path = '/Users/subashkhanal/Desktop/BMI633/ADNI_Genetics/Genomics/data/'

In [41]:
N = 50
#Gene ranking based on ttest
ttest = read_csv(os.path.join(data_path,'t_test_0.05_geneExpr_Unfiltered_bl.csv')).sort_values('CN_AD')
important_probes = ttest.sort_values('CN_AD'+'_c')['Gene'][0:N] #suffix _c to use the FDR corrected p values

#Gene Expression Data
Gene_expr = read_csv(os.path.join(data_path,'Unfiltered_gene_expr_dx.csv'),low_memory=False)
Gene_expr = Gene_expr[['Unnamed: 0','AGE','PTGENDER','PTEDUCAT','DX_bl']+list(important_probes)]

In [42]:
Gene_expr

,Unnamed: 0,AGE,PTGENDER,PTEDUCAT,DX_bl,11761978_at_ARHGEF12,11754382_a_at_SMIM5,11755705_a_at_SMIM5,11758611_s_at_TRIM10,11724079_s_at_E2F2,...,11743992_at_UBE2O,11716563_s_at_AIMP2,11744861_a_at_DHRS7,AFFX-r2-Bs-phe-3_at_nan,11750558_s_at_ZNF271,11731447_x_at_SNCA,11734736_a_at_FAR1,11719828_x_at_QPCT,11750143_a_at_PIGZ,11732501_a_at_TMOD1
0,116_S_1249,70.8,Female,15.0,CN,4.573,7.203,7.587,7.559,9.251,...,5.582,7.690,10.620,8.780,3.804,9.955,3.541,9.176,2.229,6.533
1,037_S_4410,69.1,Female,14.0,CN,4.969,5.378,5.535,5.297,8.335,...,4.828,7.919,10.619,8.722,4.303,9.469,4.960,8.711,1.998,5.096
2,006_S_4153,79.3,Male,20.0,AD,4.805,6.094,6.355,6.496,8.940,...,5.132,7.584,10.609,8.995,3.459,8.780,3.147,9.201,2.223,5.210
3,116_S_1232,72.1,Female,14.0,CN,6.287,8.110,8.376,9.000,10.421,...,6.852,7.967,10.519,9.449,3.327,11.047,2.753,8.706,2.570,8.093
4,099_S_4205,81.4,Female,18.0,EMCI,3.969,5.913,6.209,5.448,8.424,...,4.955,7.649,10.623,8.852,3.480,8.790,3.891,9.160,2.310,4.909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,009_S_2381,69.1,Male,16.0,EMCI,6.367,8.106,8.419,8.372,10.166,...,5.808,8.165,10.715,10.313,3.373,10.318,3.506,9.164,2.408,6.686
740,053_S_4557,83.3,Female,12.0,EMCI,5.978,7.691,8.061,8.452,9.639,...,5.818,7.985,10.610,9.692,3.722,10.042,2.983,8.956,2.404,6.847
741,073_S_4300,80.6,Female,16.0,LMCI,5.755,7.884,8.089,8.792,10.004,...,6.842,8.074,10.284,9.392,3.905,10.786,4.011,8.702,2.359,7.454
742,041_S_4014,80.7,Male,16.0,CN,5.293,7.708,8.010,7.422,9.114,...,5.800,8.107,10.337,10.223,3.286,9.392,3.142,8.212,2.619,5.389


In [43]:
df = Gene_expr.dropna(subset=['PTGENDER'])
print(Counter(df.DX_bl))
df = df[(df['DX_bl']=='AD') | (df['DX_bl']=='CN')]
y = df.DX_bl
df = df.drop(columns=['Unnamed: 0','DX_bl'])
df

Counter({'CN': 260, 'LMCI': 225, 'EMCI': 215, 'AD': 43})


,AGE,PTGENDER,PTEDUCAT,11761978_at_ARHGEF12,11754382_a_at_SMIM5,11755705_a_at_SMIM5,11758611_s_at_TRIM10,11724079_s_at_E2F2,11752515_a_at_ODC1,11750580_x_at_ARHGEF12,...,11743992_at_UBE2O,11716563_s_at_AIMP2,11744861_a_at_DHRS7,AFFX-r2-Bs-phe-3_at_nan,11750558_s_at_ZNF271,11731447_x_at_SNCA,11734736_a_at_FAR1,11719828_x_at_QPCT,11750143_a_at_PIGZ,11732501_a_at_TMOD1
0,70.8,Female,15.0,4.573,7.203,7.587,7.559,9.251,8.499,4.948,...,5.582,7.690,10.620,8.780,3.804,9.955,3.541,9.176,2.229,6.533
1,69.1,Female,14.0,4.969,5.378,5.535,5.297,8.335,8.261,4.934,...,4.828,7.919,10.619,8.722,4.303,9.469,4.960,8.711,1.998,5.096
2,79.3,Male,20.0,4.805,6.094,6.355,6.496,8.940,8.294,4.903,...,5.132,7.584,10.609,8.995,3.459,8.780,3.147,9.201,2.223,5.210
3,72.1,Female,14.0,6.287,8.110,8.376,9.000,10.421,8.859,6.282,...,6.852,7.967,10.519,9.449,3.327,11.047,2.753,8.706,2.570,8.093
8,84.1,Male,19.0,5.102,7.619,7.938,7.256,9.904,8.538,5.469,...,6.191,8.175,10.621,10.572,3.225,9.509,2.883,8.694,2.319,6.090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,72.9,Male,16.0,5.065,8.365,8.638,8.293,9.558,8.960,5.385,...,6.196,8.154,10.425,10.173,3.025,10.702,2.896,8.400,2.279,7.605
731,73.0,Female,16.0,4.028,6.700,6.944,7.562,9.298,7.921,4.313,...,5.440,7.756,10.620,9.253,3.700,10.349,3.410,9.109,2.180,6.971
732,72.6,Male,12.0,5.631,7.049,7.388,8.292,10.198,9.062,5.889,...,6.036,8.185,10.586,9.575,3.311,10.527,4.563,8.842,2.601,6.792
734,84.3,Male,17.0,4.563,7.017,7.071,7.277,9.013,8.695,4.524,...,6.030,8.120,10.547,10.113,3.392,9.657,2.563,8.893,2.389,5.906


In [49]:
num_col_index = [0,2] + list(range(3,3+N))
cat_col_index = [1]
num_col = list(df.iloc[:,num_col_index].columns)
cat_col = list(df.iloc[:,cat_col_index].columns)
df[num_col] = df[num_col].fillna(value=df[num_col].mean())

#df.isnull().sum(axis = 0)

In [45]:
df=df.dropna()
df.columns[df.isna().any()].tolist()

[]

In [48]:
import os
def train_scaler(data,num_col_index):
    scaled_features = data.copy()
    features = scaled_features.iloc[:,num_col_index]
    scaler = StandardScaler().fit(features.values)
    features = scaler.transform(features.values)
    scaled_features.iloc[:,num_col_index] = features
    return scaler, scaled_features
    
def test_scaler(scaler,data,num_col_index):
    scaled_features = data.copy()
    features = scaled_features.iloc[:,num_col_index]
    features = scaler.transform(features.values)
    scaled_features.iloc[:,num_col_index] = features
    
    return scaled_features

# prepare categorical input data
def prepare_cat(X):
    oe = OneHotEncoder()
    oe.fit(X)
    X_enc = oe.transform(X)
    return X_enc
 
# prepare target
def prepare_targets(y):
    le = LabelEncoder()
    le.fit(y)
    y = le.transform(y)
    return y

def data_prep(X_train,y_train,X_test,y_test,num_col_index, cat_col_index,preprocessing='none'):
    
    if preprocessing == 'none':
        # One hot encoding of categorical variable
        combined_data = pd.concat([X_train.iloc[:,cat_col_index], X_test.iloc[:,cat_col_index]], ignore_index=True)
        combined_data_cat = prepare_cat(combined_data)
        X_train_cat = combined_data_cat[0:len(X_train),:]
        X_test_cat = combined_data_cat[len(X_train):,:]

        #Concatenation of numerical and categorical inputs
        X_train_final = hstack((X_train_cat,X_train.iloc[:,num_col_index]))
        X_test_final = hstack((X_test_cat,X_test.iloc[:,num_col_index]))


        # Binarization of target label
        y_train_final = prepare_targets(y_train)
        y_test_final = prepare_targets(y_test)


        return X_train_final,y_train_final,X_test_final,y_test_final
        
    else:
        #Oversample the minority class 
        smt = SMOTENC(categorical_features =cat_col_index, 
                    random_state=123,
                    k_neighbors=7,
                    sampling_strategy=float(0.6)
                         )
        X_trainres,y_trainres = smt.fit_resample(X_train,y_train) #Oversampled training set

        # Standard Scaling of numeric variable
        scaler, X_trainres_scaled = train_scaler(X_trainres,num_col_index)
        X_test_scaled = test_scaler(scaler,X_test,num_col_index)

        # One hot encoding of categorical variable
        combined_data = pd.concat([X_trainres_scaled.iloc[:,cat_col_index], X_test_scaled.iloc[:,cat_col_index]], ignore_index=True)
        combined_data_cat = prepare_cat(combined_data)
        X_train_cat = combined_data_cat[0:len(X_trainres_scaled),:]
        X_test_cat = combined_data_cat[len(X_trainres_scaled):,:]

        #Concatenation of numerical and categorical inputs
        X_train_final = hstack((X_train_cat,X_trainres_scaled.iloc[:,num_col_index]))
        X_test_final = hstack((X_test_cat,X_test_scaled.iloc[:,num_col_index]))


        # Binarization of target label
        y_train_final = prepare_targets(y_trainres)
        y_test_final = prepare_targets(y_test)


        return X_train_final,y_train_final,X_test_final,y_test_final


def classifier(name,n_estimators=100):
    if name == 'RandomForestClassifier':
        return RandomForestClassifier(n_estimators=n_estimators,criterion='gini',class_weight='balanced',random_state=1)
    
    if name == 'GradientBoosting':
        return GradientBoostingClassifier(n_estimators=n_estimators,random_state=1)
    
    else:
        raise NotImplementedError("This classifier is not implemented yet. Please choose from [RandomForest, GradientBoosting]")

        
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=1)
X = df
acc = []
f1 = []
auc = []
imp = []
for train, test in cv.split(df, y):
    X_train = X.iloc[train]
    y_train = y.iloc[train]
    
    X_test = X.iloc[test]
    y_test = y.iloc[test]

    X_train_final,y_train_final,X_test_final,y_test_final = data_prep(X_train,y_train,X_test,y_test,num_col_index, cat_col_index)
    n_estimators = 1000
    model = classifier('GradientBoosting', n_estimators)
    probas_ = model.fit(X_train_final, y_train_final).predict_proba(X_test_final)
    y_pred = model.predict(X_test_final)
    acc.append(balanced_accuracy_score(y_test_final, y_pred))
    f1.append(f1_score(y_test_final, y_pred, average='macro'))
    auc.append(roc_auc_score(y_test_final, probas_[:, 1]))	    	
    imp.append(model.feature_importances_)
print(sum(acc)/len(acc))
print(sum(auc)/len(auc))

0.6545726495726495
0.8198290598290601


# Conclusion

The StandardScaler() should be correctly used, It caused significant data leakage issue.

SMOTENC and StandardScaler() combined rather worsen the results